In [13]:
from numbers import Number
import numpy as np
import matplotlib.pyplot as plt
import onix.models.hyperfine as hyperfine

In [3]:
ground = hyperfine.states["7F0"]
excited = hyperfine.states["5D0"]
optical_hyperfine_probabilities = hyperfine.optical_hyperfine_probabilities

In [20]:
def print_table(lists, ndecimals=3, column_size=8):
    for list in lists:
        line = ""
        for kk in list:
            if isinstance(kk, float):
                line += f"{kk:{column_size}.{ndecimals}f}"
            elif isinstance(kk, int):
                line += f"{kk:{column_size}.{ndecimals}f}"
            else:
                line += f"{kk:>{column_size}}"
        print(line)

### Optical hyperfine probabilities

In [38]:
table = [["g\\e"] + excited.states]
for g_state in ground.states:
    row = [g_state]
    for e_state in excited.states:
        row.append(optical_hyperfine_probabilities[g_state][e_state])
    table.append(row)
print("580 nm hyperfine excitation probabilities:")
print_table(table)

580 nm hyperfine excitation probabilities:
     g\e       c      c'       b      b'       a      a'
       a   0.866   0.004   0.094   0.004   0.031   0.001
      a'   0.004   0.866   0.004   0.094   0.001   0.031
       b   0.105   0.009   0.690   0.026   0.157   0.012
      b'   0.009   0.105   0.026   0.690   0.012   0.157
       c   0.007   0.009   0.163   0.022   0.799   0.000
      c'   0.009   0.007   0.022   0.163   0.000   0.799


### Ground M1 matrix elements

In [42]:
M1_elements = ground.m1_elements()
for kk, polarization in enumerate(["sigma+", "sigma-", "pi"]):
    table = [["i\\f"] + ground.states]
    for g_state in ground.states:
        row = [g_state]
        for g_state_1 in ground.states:
            row.append(abs(M1_elements[kk][g_state][g_state_1]))
        table.append(row)
    print(f"Polarization: {polarization}. Unit: 2π MHz/T.")
    print_table(table, ndecimals=2)
    print()

Polarization: sigma+. Unit: 2π MHz/T.
     i\f       a      a'       b      b'       c      c'
       a   10.88    0.06    5.23    1.56    1.30    0.49
      a'    0.49   10.88    1.25    1.66    0.84    0.36
       b    1.66    1.56    5.25    1.11    6.64    2.02
      b'    1.25    5.23    4.99    5.25    2.16    0.85
       c    0.36    0.49    0.85    2.02    5.33    0.99
      c'    0.84    1.30    2.16    6.64    2.25    5.33

Polarization: sigma-. Unit: 2π MHz/T.
     i\f       a      a'       b      b'       c      c'
       a   10.88    0.49    1.66    1.25    0.36    0.84
      a'    0.06   10.88    1.56    5.23    0.49    1.30
       b    5.23    1.25    5.25    4.99    0.85    2.16
      b'    1.56    1.66    1.11    5.25    2.02    6.64
       c    1.30    0.84    6.64    2.16    5.33    2.25
      c'    0.49    0.36    2.02    0.85    0.99    5.33

Polarization: pi. Unit: 2π MHz/T.
     i\f       a      a'       b      b'       c      c'
       a    3.44    0.69    9.80 